In [3]:
import numpy as np
import pandas as pd
from data_loader import load_names_from_web, holdout_split, year_split, births_per_year
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, accuracy_score
from scipy.stats import kendalltau

In [4]:
dfraw = load_names_from_web(category='national', hide_pre_1937=True, use_existing_files=True)
# traintestval, holdout = year_split(dfraw, holdout_size=30)
trainval, test = year_split(dfraw, holdout_size=30)
births_by_year = births_per_year()

In [5]:
sorted(trainval['year'].unique())

[1937,
 1938,
 1939,
 1940,
 1941,
 1942,
 1943,
 1944,
 1945,
 1946,
 1947,
 1948,
 1949,
 1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992]

In [6]:
def get_all_known_names(data):

    names = data.groupby(['state', 'name', 'M/F']).size().reset_index()
    names = names[['state', 'name', 'M/F']]
    return names

In [7]:
def select_top_names(data, first_year_to_predict, cutoff=10):
    '''
    Select names that have had at least one year at or above the cutoff count?
    Could also try to redo this to do the cutoff for the most recent known year's data
    '''

    data_noleak = data[data['year'] < first_year_to_predict]
    names = data_noleak[data_noleak['count'] >= cutoff][['state', 'name', 'M/F']].drop_duplicates()
    # display(names)
    data = names.merge(data, how='left', on=['state', 'name', 'M/F'])
    # display(data)

    return data

In [8]:
def classify(df):

    '''
    PEAKS
    '''

    # peaks = df.loc[df.groupby(['state', 'name', 'M/F'])['count'].idxmax()][['state', 'name', 'M/F', 'year']]
    # peaks = peaks.rename(columns={'year': 'peak_year'})
    # df = df.merge(peaks, how='left', on=['state', 'name', 'M/F'])

    df = df.sort_values(by=['year', 'state', 'name', 'M/F'])
    all_names = df.drop(columns='count').drop_duplicates(subset=['state', 'name', 'M/F']).rename(columns={'year': 'first_year'})
    # display(all_names)
    all_years = range(1937, 2023)
    all_names = all_names.assign(key=1)
    all_years = pd.DataFrame({'year': all_years, 'key': 1})
    all_names_all_years = pd.merge(all_names, all_years, on='key').drop(columns='key')
    df = all_names_all_years.merge(df, how='left', on=['state', 'name', 'M/F', 'year'])
    df['count'] = df['count'].fillna(0)
    df = df[df['year'] >= df['first_year']]

    df['pct'] = df.apply(lambda row: row['count'] / births_by_year.at[row['year'], row['M/F']], axis=1)

    peaks = df.loc[df.groupby(['state', 'name', 'M/F'])['pct'].idxmax()][['state', 'name', 'M/F', 'year', 'pct']]
    peaks = peaks.rename(columns={'year': 'peak_pct_year', 'pct': 'peak_pct'})
    df = df.merge(peaks, how='left', on=['state', 'name', 'M/F'])

    df['has_peaked'] = df.apply(lambda row: 1 if row['year'] >= row['peak_pct_year'] else 0, axis=1)
    return df

In [10]:
example = classify(trainval)
example[(example['peak_pct_year'] == 1984) & (example['year'] == 1983)].sort_values(by='count', ascending=False).head(10)

,state,first_year,name,M/F,year,count,pct,peak_pct_year,peak_pct,has_peaked
127584,US,1937,Christopher,M,1983,59340.0,0.031839,1984,0.031982,0
2626,US,1937,Adam,M,1983,23489.0,0.012603,1984,0.012756,0
687014,US,1937,Stephanie,F,1983,22331.0,0.012478,1984,0.012768,0
805443,US,1938,Dustin,M,1983,9680.0,0.005194,1984,0.005541,0
451374,US,1937,Lindsey,F,1983,8681.0,0.004851,1984,0.004994,0
16558,US,1937,Alicia,F,1983,7491.0,0.004186,1984,0.004337,0
494976,US,1937,Marcus,M,1983,5634.0,0.003023,1984,0.003196,0
1789591,US,1960,Latoya,F,1983,3151.0,0.001761,1984,0.002801,0
1199093,US,1943,Sheena,F,1983,2634.0,0.001472,1984,0.001995,0
379392,US,1937,Joanna,F,1983,2120.0,0.001185,1984,0.001847,0


In [11]:
def evaluate(predictor, data_held_out, first_year_to_predict, metric='accuracy'):

    # display('data_held_out:')
    # display(data_held_out)

    # most_recent_year = data_held_out['year'].max()
    most_recent_year = first_year_to_predict - 1
    
    # years_to_predict = range(first_year_to_predict, most_recent_year+1)

    # only allow the model to see data from before the year to predict
    historical_data = data_held_out[data_held_out['year'] < first_year_to_predict]

    # display('historical_data:')
    # display(historical_data)

    # get our model's predictions
    predictions = predictor.predict(historical_data)

    all_known_names = get_all_known_names(historical_data)

    # now we need to know the "answers" -- what the real classes are, based on information in the "future"
    data_held_out = classify(data_held_out)
    data_held_out = data_held_out[data_held_out['year'] == most_recent_year]
    # print(f'Predictions for {year_to_predict}:')

    # names_to_predict = all_known_names.copy()
    # names_to_predict['year'] = year_to_predict
    # display(names_to_predict)

    observed = all_known_names.merge(data_held_out, how='left', on=['state', 'name', 'M/F'])

    observed['y'] = observed['has_peaked'].fillna(1) # if nothing for this name for this year, it has already peaked, so 1
    # display(observed)

    score_df = observed.merge(predictions, how='left', on=['state', 'name', 'M/F'], suffixes=('_true', '_pred'))
    # display(score_df)

    y_true = score_df['y_true']
    y_pred = score_df['y_pred']

    '''
    if metric == 'msle':
        loss = mean_squared_log_error(y_true, y_pred)
        print(f'Loss: {loss}')

    if metric == 'rank':
        y_true = y_true.rank()
        y_pred = y_pred.rank()
        score = np.sum(np.abs(y_true-y_pred))/(len(y_true)*(len(y_true)-1))
        print(f'Score: {score}')

    if metric == 'rank_mae':
        y_true = y_true.rank()
        y_pred = y_pred.rank()
        loss = mean_absolute_error(y_true, y_pred)
        print(f'Loss: {loss}')

    if metric == 'kendalltau':
        y_true = y_true.rank()
        y_pred = y_pred.rank()
        tau, _ = kendalltau(y_true, y_pred)
        print(f'Tau: {tau}')
    '''

    if metric == 'accuracy':
        score = accuracy_score(y_true, y_pred)
        print(f'Score: {score}')
    
    '''
    top_F_true = score_df[score_df['M/F'] == 'F'][['name', 'y_true']].sort_values(by='y_true', ascending=False).reset_index(drop=True)
    top_F_pred = score_df[score_df['M/F'] == 'F'][['name', 'y_pred']].sort_values(by='y_pred', ascending=False).reset_index(drop=True)
    top_M_true = score_df[score_df['M/F'] == 'M'][['name', 'y_true']].sort_values(by='y_true', ascending=False).reset_index(drop=True)
    top_M_pred = score_df[score_df['M/F'] == 'M'][['name', 'y_pred']].sort_values(by='y_pred', ascending=False).reset_index(drop=True)
    top = pd.concat([top_F_true, top_F_pred, top_M_true, top_M_pred], axis=1, ignore_index=True)
    '''
    display(score_df)

In [14]:
class DummyClassifier():

    def __init__(self, strategy='after_peak'):
        self.strategy = strategy
    
    def predict(self, historical_data):

        all_known_names = get_all_known_names(historical_data)

        most_recent_year = historical_data['year'].max()

        predictions = all_known_names.copy()

        if self.strategy == 'constant':
            predictions['y'] = 1
        elif self.strategy == 'after_peak':
            historical_data = classify(historical_data)
            most_recent_year_data = historical_data[historical_data['year'] == most_recent_year].drop(columns=['year'])
            predictions = predictions.merge(most_recent_year_data, how='left', on=['state', 'name', 'M/F'])
            predictions['peak_pct_year'] = predictions['peak_pct_year'].fillna(0)
            predictions['y'] = predictions['peak_pct_year'].apply(lambda x: 1 if x < most_recent_year else 0)
            # display(predictions)

        return predictions

In [15]:
first_year_to_predict = 1993
cutoff = 100
# data_to_fit = select_top_names(trainval, first_year_to_predict=first_year_to_predict, cutoff=0)
data_to_eval = select_top_names(test, first_year_to_predict=first_year_to_predict, cutoff=cutoff)
evaluate(predictor=DummyClassifier(strategy='after_peak'), data_held_out=data_to_eval, first_year_to_predict=first_year_to_predict, metric='accuracy')

Score: 0.8235427022141889


,state,name,M/F,first_year_true,year,count_true,pct_true,peak_pct_year_true,peak_pct_true,has_peaked_true,y_true,first_year_pred,count_pred,pct_pred,peak_pct_year_pred,peak_pct_pred,has_peaked_pred,y_pred
0,US,Aaron,F,1937,1992,91.0,0.000045,1976,0.000098,1,1,1937,91.0,0.000045,1976,0.000098,1,1
1,US,Aaron,M,1937,1992,14509.0,0.006912,1981,0.007890,1,1,1937,14509.0,0.006912,1981,0.007890,1,1
2,US,Abbey,F,1938,1992,431.0,0.000215,1999,0.000365,0,0,1938,431.0,0.000215,1990,0.000247,1,1
3,US,Abbie,F,1937,1992,260.0,0.000130,2003,0.000268,0,0,1937,260.0,0.000130,1981,0.000163,1,1
4,US,Abby,F,1937,1992,1081.0,0.000539,2003,0.001021,0,0,1937,1081.0,0.000539,1983,0.000732,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4421,US,Zelma,F,1937,1992,8.0,0.000004,1937,0.000173,1,1,1937,8.0,0.000004,1937,0.000173,1,1
4422,US,Zena,F,1937,1992,52.0,0.000026,1964,0.000113,1,1,1937,52.0,0.000026,1964,0.000113,1,1
4423,US,Zina,F,1937,1992,21.0,0.000010,1964,0.000405,1,1,1937,21.0,0.000010,1964,0.000405,1,1
4424,US,Zoe,F,1937,1992,982.0,0.000490,2012,0.003334,0,0,1937,982.0,0.000490,1992,0.000490,1,0


In [18]:
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier

class MyPredictor():

    def __init__(self):
        
        # params
        # cols_to_keep = ['this_year', 'M/F', 'sum', 'median_age', 'thisyear_count', 'diff',] # best overall?
        # cols_to_keep = ['this_year', 'M/F', 'sum', 'median_age', 'thisyear_count', 'diff', 'first_letter_1_pct', 'first_letter_2_pct', 'first_letter_3_pct'] # first letter pct change helps in some years
        # cols_to_keep = ['this_year', 'M/F', 'sum', 'median_age', 'thisyear_count', 'diff', 'thisyear_count_opp']
        cols_to_keep = ['M/F', 'sum', 'median_age', 'thisyear_count', 'diff', 'thisyear_count_opp', 'diff2', 'shift', 'pct_change', 'accel', 'after_peak', 'years_since_peak', 'first_letter_1_pct', 'first_letter_2_pct', 'first_letter_3_pct']
        categorical_features = ['M/F', 'after_peak']
        max_leaf_nodes = 31 # 16
        max_iter = 100 # 100
        loss = 'log_loss' # abs better than default

        categorical_features = [True if f in categorical_features else False for f in cols_to_keep]
        # print(categorical_features)
        
        self.pipe = make_pipeline(
            ColumnTransformer(
                transformers=[
                    # ('category_encoder', LabelEncoder(), categorical_features),
                    ('cols_to_keep', 'passthrough', cols_to_keep),
                ], remainder='drop'),
            HistGradientBoostingClassifier(
                random_state=0,
                categorical_features=categorical_features,
                max_leaf_nodes=max_leaf_nodes,
                max_iter=max_iter,
                loss=loss
            )
        )

        self.gender_encoding = {'M': 0, 'F': 1}

    def preprocess(self, df, latest_known_year):

        # print(f'Latest known year: {latest_known_year}')

        # find median age of people with name, 
        # total born with that name,
        # and latest year's count

        df = df.copy()

        df = classify(df)
        df['after_peak'] = df['peak_pct_year'].apply(lambda x: 1 if x < latest_known_year else 0)

        df = df.sort_values(by='year')
        df['cumsum'] = df.groupby(['state', 'name', 'M/F'])['count'].cumsum()
        df['sum'] = df.groupby(['state', 'name', 'M/F'])['count'].transform('sum')

        df['diff'] = df.groupby(['state', 'name', 'M/F'])['count'].diff()
        df['diff2'] = df.groupby(['state', 'name', 'M/F'])['count'].diff(2)
        df['shift'] = df.groupby(['state', 'name', 'M/F'])['count'].shift()
        df['pct_change'] = df.groupby(['state', 'name', 'M/F'])['count'].pct_change()
        df['accel'] = df.groupby(['state', 'name', 'M/F'])['diff'].diff()

        percentage_of_total_per_year = {}
        percentage_change_per_year = {}

        def first_letters(df, n, percentage_of_total_per_year, percentage_change_per_year):
            df['first_letter_'+str(n)] = df['name'].str[0:n].str.lower()
            total_names_per_year = df.groupby(['year', 'state', 'M/F'])['count'].sum()
            letter_names_per_year = df.groupby(['year', 'state', 'M/F', 'first_letter_'+str(n)])['count'].sum()
            percentage_of_total_per_year[n] = (letter_names_per_year / total_names_per_year).rename('first_letter_'+str(n)+'_pct')
            # display(percentage_of_total_per_year)
            percentage_change_per_year[n] = percentage_of_total_per_year[n].groupby(['state', 'M/F', 'first_letter_'+str(n)]).pct_change().rename('first_letter_'+str(n)+'_pct_change')
            # display(percentage_change_per_year)
            return df
        
        for n in range(1, 4):
            df = first_letters(df, n, percentage_of_total_per_year, percentage_change_per_year)
        
        # display(df[(df['name'] == 'Maximus')])

        medians = df[df['cumsum'] >= df['sum']/2]
        medians = medians.drop_duplicates(subset=['state', 'name', 'M/F'], keep='first')
        medians['median_age'] = latest_known_year - medians['year']
        # display(medians[medians['name'] == 'Madison'])

        medians = medians.drop(['count', 'cumsum', 'diff', 'shift', 'pct_change', 'accel', 'diff2', 'after_peak'], axis=1)

        thisyear = df[df['year'] == latest_known_year][['state', 'name', 'M/F', 'count', 'diff', 'shift', 'pct_change', 'accel', 'diff2', 'after_peak']].rename(columns={'count': 'thisyear_count'})
        # thisyear = thisyear.merge(percentage_of_total_per_year, how='left', on=['year', 'state', 'M/F', 'first_letter_1'])
        # thisyear = thisyear.merge(percentage_change_per_year, how='left', on=['year', 'state', 'M/F', 'first_letter_1'])
        # thisyear = thisyear.rename(columns={'year': 'this_year'})

        sex_counts = thisyear.groupby(['state', 'name', 'M/F'])['thisyear_count'].sum()
        thisyear_swapped = thisyear.copy()
        thisyear_swapped['M/F'] = thisyear_swapped['M/F'].replace({'M': 'F', 'F': 'M'})
        thisyear_swapped = thisyear_swapped.merge(sex_counts, how='left', on=['state', 'name', 'M/F'], suffixes=('', '_opp'))
        thisyear_swapped['M/F'] = thisyear_swapped['M/F'].replace({'M': 'F', 'F': 'M'})
        thisyear_swapped['thisyear_count_opp'] = thisyear_swapped['thisyear_count_opp'].fillna(0)
        thisyear = thisyear_swapped
        # display(thisyear)

        df = medians.merge(thisyear, how='left', on=['state', 'name', 'M/F']).rename(columns={'year': 'median_year'})
        # df2[['thisyear_count']] = df2[['thisyear_count']].fillna(0) # might want to shift this to 2 and fill in 2s for missing years? or maybe not

        # this is sort of a rough assumption that if a row didn't exist for this year, not only is the count 0, but so is the diff, shift and pct_change. not always true if the prior year had a count, but often true
        df[['thisyear_count', 'diff', 'shift', 'pct_change', 'accel', 'diff2']] = df[['thisyear_count', 'diff', 'shift', 'pct_change', 'accel', 'diff2']].fillna(0)

        df['after_peak'] = df['after_peak'].fillna(1)
        df['years_since_peak'] = latest_known_year - df['peak_pct_year']

        df['year'] = latest_known_year

        for n in range(1, 4):
            df = df.merge(percentage_of_total_per_year[n], how='left', on=['year', 'state', 'M/F', 'first_letter_'+str(n)])
            df = df.merge(percentage_change_per_year[n], how='left', on=['year', 'state', 'M/F', 'first_letter_'+str(n)])
            df['first_letter_'+str(n)+'_pct'] = df['first_letter_'+str(n)+'_pct'].fillna(0)
            df['first_letter_'+str(n)+'_pct_change'] = df['first_letter_'+str(n)+'_pct_change'].fillna(0)

        # display(df2)
        # display(df2.groupby(['state','name','M/F']).ngroups)

        # change M/F to 0/1 so it works with various models
        # (even HistGradientBoostingRegressor, which accepts categorical values,
        # still needs those values to be numbers not strings)
        df['M/F'] = df['M/F'].map(self.gender_encoding)

        return df
    
    def fit(self, historical_data, first_year_to_predict, years_to_fit=1, weight_decay=0.99):
        # first, we do need to find our y
        historical_data = classify(historical_data)
        
        # first things first, we don't want to know about future data
        historical_data = historical_data[historical_data['year'] < first_year_to_predict]
        # at this point the data we don't want to know should be inaccessible

        # maybe there is a better way to do the following
        # but basically we just want the original columns for X and one new labeled column for y
        X_orig = historical_data[['state', 'year', 'name', 'M/F', 'count']]
        y_orig = historical_data[['state', 'year', 'name', 'M/F', 'has_peaked']]
        y_orig = y_orig.rename(columns={'has_peaked': 'y'})
        y_orig['M/F'] = y_orig['M/F'].map(self.gender_encoding)

        X_all = pd.DataFrame()
        y_all = pd.Series()

        # each year_to_fit is the year that's essentially our y for that loop
        for year_to_fit in range(first_year_to_predict - years_to_fit, first_year_to_predict):

            # now we "know" even less for X
            X = X_orig[X_orig['year'] <= year_to_fit]
            # y = historical_data[historical_data['year'] == year_to_fit]

            X = self.preprocess(X, latest_known_year=year_to_fit)
            # y = y[['state', 'name', 'M/F', 'count']].rename(columns={'count': 'y'})

            # display(X)

            data = X.merge(y_orig, how='left', on=['state', 'year', 'name', 'M/F'])
            data['y'] = data['y'].fillna(1) # if nothing for this name for this year, it has already peaked, so 1

            # assert data['y'].isna().any() == False

            # display(data)

            X = data.drop(columns=['y'])
            y = data['y']
            X['sample_weight'] = weight_decay ** (first_year_to_predict - year_to_fit)

            X_all = pd.concat([X_all, X], ignore_index=True)
            y_all = pd.concat([y_all, y], ignore_index=True)
        
        temp = X_all.copy()
        temp['y'] = y_all
        display(temp)
        display(temp.columns)

        sample_weights = X_all['sample_weight']
        X_all = X_all.drop(columns=['sample_weight'])

        self.pipe.fit(X_all, y_all, **{'histgradientboostingclassifier__sample_weight': sample_weights})
        # this seems like a silly way to pass params to individual steps of the pipeline, but it's true. See: https://stackoverflow.com/questions/36205850/sklearn-pipeline-applying-sample-weights-after-applying-a-polynomial-feature-t

    def predict(self, historical_data):

        # all_known_names = get_all_known_names(historical_data)

        # predictions = []

        # display('historical_data in predict:')
        # display(historical_data)
        # display('years_to_predict:')
        # display(years_to_predict)

        most_recent_year = historical_data['year'].max()

        # predictions = all_known_names.copy()

        # most_recent_year_data = historical_data[historical_data['year'] == most_recent_year]
        # predictions = predictions.merge(most_recent_year_data, how='left', on=['state', 'name', 'M/F'])
        # display(predictions)

        # display('historical_data in predict loop:')
        # display(historical_data)

        df = self.preprocess(historical_data, latest_known_year=most_recent_year)
        # df = self.preprocess(historical_data, years_to_predict[0] - 1)

        df = df[df['year'] == most_recent_year]

        df['y'] = self.pipe.predict(df)

        # df['year'] = year_to_predict
        # display(df)

        # if we want to simply, do the following; 
        # but for now, might be useful to see all data displayed.
        # df = df[['state', 'year', 'name', 'M/F', 'y']]

        predictions = df.copy()

        '''
        assumed_new_year_of_historical_data = df[['state', 'year', 'name', 'M/F', 'y']].rename(columns={'y': 'count'})
        assumed_new_year_of_historical_data['M/F'] = assumed_new_year_of_historical_data['M/F'].map({v: k for k, v in self.gender_encoding.items()})
        historical_data = pd.concat([historical_data, assumed_new_year_of_historical_data], ignore_index=True)
        '''

        # predictions = pd.concat(predictions, ignore_index=True)

        # we have to reverse the mapping to send our predictions
        # (at least the way we currently have it set up)
        predictions['M/F'] = predictions['M/F'].map({v: k for k, v in self.gender_encoding.items()})

        # predictions.loc[predictions['y'] < 4.5, 'y'] = 2

        display(predictions)
        # display(predictions[predictions['y'] < 4.5])

        return predictions

In [19]:
first_year_to_predict = 1993
years_to_predict_in_fitting = 15
years_to_fit_in_fitting = 30
name_usage_cutoff = 100
my_predictor = MyPredictor()
data_to_fit = select_top_names(trainval, first_year_to_predict=first_year_to_predict, cutoff=name_usage_cutoff)
data_to_eval = select_top_names(test, first_year_to_predict=first_year_to_predict, cutoff=name_usage_cutoff)
my_predictor.fit(historical_data=data_to_fit, first_year_to_predict=first_year_to_predict-years_to_predict_in_fitting, years_to_fit=years_to_fit_in_fitting)
evaluate(predictor=my_predictor, data_held_out=data_to_eval, first_year_to_predict=first_year_to_predict, metric='accuracy')

,state,first_year,name,M/F,median_year,pct,peak_pct_year,peak_pct,has_peaked,sum,...,years_since_peak,year,first_letter_1_pct,first_letter_1_pct_change,first_letter_2_pct,first_letter_2_pct_change,first_letter_3_pct,first_letter_3_pct_change,sample_weight,y
0,US,1937,Nikolas,0,1937,0.000005,1937,0.000005,1,6.0,...,11,1948,0.007727,-0.017184,0.001596,-0.014660,0.000000,0.000000,0.7397,0
1,US,1937,Claribel,1,1937,0.000009,1937,0.000009,1,18.0,...,11,1948,0.094005,0.012204,0.004255,-0.040156,0.004063,-0.036637,0.7397,0
2,US,1937,Jerimiah,0,1937,0.000005,1937,0.000005,1,5.0,...,11,1948,0.168296,-0.021434,0.019686,-0.016231,0.012099,-0.057831,0.7397,0
3,US,1937,Gage,0,1937,0.000005,1937,0.000005,1,10.0,...,11,1948,0.055321,-0.015659,0.018627,-0.010274,0.000000,0.000000,0.7397,0
4,US,1937,Adonis,0,1937,0.000005,1937,0.000005,1,10.0,...,11,1948,0.035509,0.007770,0.000517,0.022720,0.000133,0.257483,0.7397,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112033,US,1975,Skyler,1,1977,0.000006,1977,0.000006,1,15.0,...,0,1977,0.100167,0.028097,0.000089,0.128263,0.000089,0.128263,0.9900,0
112034,US,1977,Kayley,1,1977,0.000003,1977,0.000003,1,5.0,...,0,1977,0.088922,0.107665,0.029219,0.070103,0.000446,-0.000831,0.9900,0
112035,US,1968,Shawnte,1,1977,0.000126,1977,0.000126,1,360.0,...,0,1977,0.100167,0.028097,0.031344,-0.019514,0.021433,0.025116,0.9900,1
112036,US,1977,Kizzy,1,1977,0.000678,1977,0.000678,1,1116.0,...,0,1977,0.088922,0.107665,0.017151,-0.013896,0.000987,inf,0.9900,1


Index(['state', 'first_year', 'name', 'M/F', 'median_year', 'pct',
       'peak_pct_year', 'peak_pct', 'has_peaked', 'sum', 'first_letter_1',
       'first_letter_2', 'first_letter_3', 'median_age', 'thisyear_count',
       'diff', 'shift', 'pct_change', 'accel', 'diff2', 'after_peak',
       'thisyear_count_opp', 'years_since_peak', 'year', 'first_letter_1_pct',
       'first_letter_1_pct_change', 'first_letter_2_pct',
       'first_letter_2_pct_change', 'first_letter_3_pct',
       'first_letter_3_pct_change', 'sample_weight', 'y'],
      dtype='object')

/Users/194627/Sites/capstone/capstone_venv/lib/python3.8/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,state,name,M/F,first_year,median_year,pct,peak_pct_year,peak_pct,has_peaked,sum,...,thisyear_count_opp,years_since_peak,year,first_letter_1_pct,first_letter_1_pct_change,first_letter_2_pct,first_letter_2_pct_change,first_letter_3_pct,first_letter_3_pct_change,y
0,US,Shelva,F,1937,1940,0.000059,1937,0.000176,1,1028.0,...,0.0,55,1992,0.092766,-0.007713,0.021895,0.022871,0.008309,-0.057600,1
1,US,Shelba,F,1937,1941,0.000104,1937,0.000312,1,1964.0,...,0.0,55,1992,0.092766,-0.007713,0.021895,0.022871,0.008309,-0.057600,1
2,US,Melvyn,M,1937,1942,0.000200,1938,0.000258,1,3013.0,...,0.0,54,1992,0.083813,-0.049057,0.000596,-0.049408,0.000450,-0.053206,1
3,US,Betty,M,1937,1944,0.000053,1939,0.000115,1,1678.0,...,296.0,53,1992,0.056593,0.029490,0.008544,-0.019945,0.000000,0.000000,1
4,US,Ima,F,1937,1944,0.000056,1937,0.000140,1,1740.0,...,0.0,55,1992,0.005141,-0.077517,0.000414,0.514381,0.000356,0.650606,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4421,US,Bronte,F,1969,1992,0.000065,1992,0.000065,1,185.0,...,0.0,0,1992,0.055148,-0.014464,0.044932,-0.012426,0.004079,0.110663,0
4422,US,Shaquille,M,1989,1992,0.000249,1992,0.000249,1,828.0,...,0.0,0,1992,0.049404,-0.023630,0.007629,-0.019393,0.006749,-0.019946,0
4423,US,Keanu,M,1990,1992,0.000050,1992,0.000050,1,184.0,...,0.0,0,1992,0.038275,-0.029571,0.020329,-0.042847,0.000256,0.297739,0
4424,US,Lexus,F,1990,1992,0.000096,1992,0.000096,1,331.0,...,0.0,0,1992,0.048600,-0.047244,0.007289,-0.021386,0.000266,0.369026,0


Score: 0.8477180298237686


,state,name,M/F,first_year_true,year_true,count,pct_true,peak_pct_year_true,peak_pct_true,has_peaked_true,...,thisyear_count_opp,years_since_peak,year_pred,first_letter_1_pct,first_letter_1_pct_change,first_letter_2_pct,first_letter_2_pct_change,first_letter_3_pct,first_letter_3_pct_change,y_pred
0,US,Aaron,F,1937,1992,91.0,0.000045,1976,0.000098,1,...,14509.0,16,1992,0.143005,-0.033572,0.000055,-0.087438,0.000055,-0.087438,1
1,US,Aaron,M,1937,1992,14509.0,0.006912,1981,0.007890,1,...,91.0,11,1992,0.091399,0.036444,0.007758,0.034494,0.007758,0.034494,1
2,US,Abbey,F,1938,1992,431.0,0.000215,1999,0.000365,0,...,0.0,2,1992,0.143005,-0.033572,0.003535,0.025855,0.001072,-0.046038,0
3,US,Abbie,F,1937,1992,260.0,0.000130,2003,0.000268,0,...,0.0,11,1992,0.143005,-0.033572,0.003535,0.025855,0.001072,-0.046038,1
4,US,Abby,F,1937,1992,1081.0,0.000539,2003,0.001021,0,...,0.0,9,1992,0.143005,-0.033572,0.003535,0.025855,0.001072,-0.046038,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4421,US,Zelma,F,1937,1992,8.0,0.000004,1937,0.000173,1,...,0.0,55,1992,0.000862,0.337240,0.000050,0.035345,0.000019,-0.039121,1
4422,US,Zena,F,1937,1992,52.0,0.000026,1964,0.000113,1,...,0.0,28,1992,0.000862,0.337240,0.000050,0.035345,0.000031,0.085496,1
4423,US,Zina,F,1937,1992,21.0,0.000010,1964,0.000405,1,...,0.0,28,1992,0.000862,0.337240,0.000013,-0.259300,0.000013,-0.259300,1
4424,US,Zoe,F,1937,1992,982.0,0.000490,2012,0.003334,0,...,0.0,0,1992,0.000862,0.337240,0.000682,0.409261,0.000682,0.409261,0
